In [3]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
pip install tensorflow


  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached h5py-3.12.1-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
  Using cached ml_dtypes-0.4.1-cp39-cp39-win_amd64.whl.metadata (20 kB)
  Using cached tensorflow_io_gcs_filesystem-0.31.0-cp39-cp39-win_amd64.whl.metadata (14 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
     ---------------------------------------- 0.0/48.7 kB ? eta -:--:--
     ------------------------------- ------ 41.0/48.7 kB 960.0 kB/s eta 0:00:01
     -------------------------------------- 48.7/48.7 kB 487.0 kB/s eta 0:00:00
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
   ---------------------------------------- 0.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [12]:
train_dir = 'C:/Users/navya/Downloads/grocery archive/GroceryStoreDataset/dataset/train'
val_dir = 'C:/Users/navya/Downloads/grocery archive/GroceryStoreDataset/dataset/val'
test_dir = 'C:/Users/navya/Downloads/grocery archive/GroceryStoreDataset/dataset/test'

In [5]:
image_paths = []
labels = []
description = []

In [22]:
def load_data_from_dir(base_dir):
    image_paths = []
    labels = []
    for category in ['Vegetables', 'Packages', 'Fruit']:
        category_path = os.path.join(base_dir, category)
        for subcategory in os.listdir(category_path):
            subcategory_path = os.path.join(category_path, subcategory)
            for filename in os.listdir(subcategory_path):
                if filename.endswith('.jpg'):
                    image_paths.append(os.path.join(subcategory_path, filename))
                    labels.append(subcategory)  # Store only the subcategory as label
    return image_paths, labels

In [23]:
train_images, train_labels = load_data_from_dir(train_dir)
val_images, val_labels = load_data_from_dir(val_dir)
test_images, test_labels = load_data_from_dir(test_dir)

In [24]:
# Check the lengths to confirm consistency
print(f"Train Images: {len(train_images)}")
print(f"Train Labels: {len(train_labels)}")
print(f"Val Images: {len(val_images)}")
print(f"Val Labels: {len(val_labels)}")
print(f"Test Images: {len(test_images)}")
print(f"Test Labels: {len(test_labels)}")

Train Images: 876
Train Labels: 876
Val Images: 111
Val Labels: 111
Test Images: 811
Test Labels: 811


In [25]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split

In [26]:
base_model = MobileNetV2(weights='imagenet', input_shape=(224, 224, 3), include_top=False)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [27]:
base_model.trainable = False

In [29]:
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])


In [30]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [31]:
train_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=True, zoom_range=0.2)
val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [32]:
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='sparse')
val_generator = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), batch_size=32, class_mode='sparse')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='sparse')

Found 2640 images belonging to 3 classes.
Found 296 images belonging to 3 classes.
Found 2485 images belonging to 3 classes.


In [33]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10, batch_size=32)

C:\Users\navya\anaconda\envs\ml\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 86s 919ms/step - accuracy: 0.8577 - loss: 0.3605 - val_accuracy: 0.9595 - val_loss: 0.1067
Epoch 2/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 74s 886ms/step - accuracy: 0.9881 - loss: 0.0373 - val_accuracy: 0.9527 - val_loss: 0.1780
Epoch 3/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 72s 868ms/step - accuracy: 0.9923 - loss: 0.0327 - val_accuracy: 0.9595 - val_loss: 0.1039
Epoch 4/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 76s 912ms/step - accuracy: 0.9938 - loss: 0.0154 - val_accuracy: 0.9561 - val_loss: 0.1099
Epoch 5/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 80s 954ms/step - accuracy: 0.9897 - loss: 0.0258 - val_accuracy: 0.9392 - val_loss: 0.2441
Epoch 6/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 77s 921ms/step - accuracy: 0.9947 - loss: 0.0116 - val_accuracy: 0.9696 - val_loss: 0.1345
Epoch 7/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 75s 901ms/step - accuracy: 0.9982 - loss: 0.0080 - val_accuracy: 0.9797 - val_loss: 0.0674
Epoch 8/10
83/83 ━━━━━━━━━━━━━━━━━━━━ 87s 1s/step - accuracy: 0.9941 - loss: 0.0170 - val_accurac

In [35]:
test_loss, test_accuracy = model.evaluate(test_generator)

78/78 ━━━━━━━━━━━━━━━━━━━━ 45s 580ms/step - accuracy: 0.9681 - loss: 0.1286


In [40]:
model.save('pantry_model.h5')

In [42]:
npm install -g http-server

SyntaxError: invalid syntax (2240296743.py, line 1)